In [12]:
import os
import util
import numpy as np
import scipy as sp
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

In [2]:
learning_rate = 0.0001
beta1 = 0.5
batch_size = 64
epi = 2
save_sample_period = 50

In [3]:
if not os.path.exists('samples'):
    os.mkdir('samples')

In [4]:
def lrelu(x,alpha = 0.2):
    return max(x*alpha,x)

In [5]:
class ConvLayer:
    def __init__(self, name, mi, mo, apply_batch_norm, filter_size = 5, stride = 2, f = tf.nn.relu):
        self.W = tf.get_variable(
            name = "W_%s"%name,
            shape = (filter_size, filter_size, mi, mo),
            initializer = tf.truncated_normal_initializer(stddev=0.02),
        )
        
        self.b = tf.get_variable(
            name = "b_%s"%name,
            shape=(mo,),
            initializer = tf.zeros_initializer(),
        )
        self.name = name,
        self.f = f,
        self.stride = stride,
        self.apply_batch_norm = apply_batch_norm,
        self.params = [self.W, self.b]
        
    def forward(self, X, reuse, is_training):
        conv_out = tf.nn.conv2d(
            X,
            self.W,
            strides = [1,self.stride,self.stride, 1],
            padding = 'SAME'
        )
        conv_out = tf.nn.bias_add(conv_out,self.b)
        
        if self.apply_batch_norm:
            conv_out = tf.contrib.layers.batch_norm(
                conv_out,
                decay = 0.9,
                updates_collection = None,
                epsilon = 1e-5,
                scale = True,
                is_training = is_training,
                reuse = reuse,
                scope = self.name,
            )
        
        return self.f(conv_out)

In [6]:
class FractioanllyStridedConvLayer:
    def __init__(self, name, mi, mo, output_shape, apply_batch_norm, filter_size = 5, stride = 2, f = tf.nn.relu):
        #here the order of mi and mo is reversed
        self.W = tf.get_variable(
            name = "W_%s"%name,
            shape = (filter_size, filter_size,mo,mi),
            initializer = tf.random_normal_initializer(stddev=0.02),
        )
        self.b = tf.get_variable(
            name = "b_%s"%name,
            shape = (mo,),
            initializer = tf.zeros_initializer(),
        )
        self.f = f,
        self.name = name,
        self.output_shape = output_shape,
        self.stride = stride,
        self.apply_batch_norm = apply_batch_norm,
        self.params = [self.W, self.b]
        
    def forward(self, X, reuse, is_training):
        conv_out = tf.nn.conv2d_transpose(
            value = X,
            filter = self.W,
            output_shape = self.output_shape,
            strides = [1,self.stride, self.stride, 1],            
        )
        
        conv_out = tf.nn.bias_add(conv_out,self.b)
        
        if self.apply_batch_norm:
            conv_out = tf.contrib.layers.batch_norm(
                conv_out,
                decay = 0.9,
                updates_collection = None,
                epsilon = 1e-5,
                scale = True,
                is_training = is_training,
                reuse = reuse,
                scope = self.name,
            )
        
        return self.f(conv_out)
            

In [7]:
class DenseLayer:
    def __init__(self, name, M1, M2, apply_batch_norm, f= tf.nn.relu):
        self.W = tf.get_variable(
            "W_%s"%name,
            shape = (M1, M2),
            initializer = tf.random_normal_initializer(stddev=0.02),
        )
        
        self.b = tf.get_variable(
            "b_%s"%name,
            shape = (M2,),
            initializer = tf.zeros_initializer(),
        )
        
        self.name = name,
        self.apply_batch_norm = apply_batch_norm,
        self.f = f,
        self.params = [self.W, self.b]
    
    def forward(self, X, reuse, is_training):
        a = tf.matmul(X, self.W) + self.b
        
        if self.apply_batch_norm:
            a = tf.contrib.layers.batch_norm(
                a,
                decay = 0.9,
                updates_collection = None,
                epsilon = 1e-5,
                scale = True,
                is_training = is_training,
                reuse = reuse,
                scope = self.name,
            )
        return self.f(a)

In [17]:
class DCGAN:
    def __init__(self, img_length, num_colors, d_sizes, g_sizes):
        self.img_length = img_length,
        self.num_colors = num_colors,
        self.latent_dims = g_sizes['z']
        
        self.X = tf.placeholder(
            tf.float32,
            shape = (None, self.img_length,self.img_length,self.num_colors),
            name = 'X'
        )
        self.Z = tf.placeholder(
            tf.float32,
            shape = (None, self.latent_dims),
            name = 'Z'
        )
        self.batch_size = tf.placeholder(
            tf.float32,
            shape=(),
            name = 'batch_size'
        )
        #build discriminator
        logits = self.build_discriminator(self.X, d_sizes)
        #build generator
        self.sample_images = self.build_generator(self.Z,g_sizes)
        
        with tf.variable_scope("discriminator") as scope:
            scope.reuse_variables()
            sample_logits = self.d_forward(self.sample_images, True)
        
        with tf.variable_scope("generator") as scope:
            scope.reuse_variables()
            self.sample_image_test = self.g_forward(self.Z, reuse = True, is_training = False)
        
        #build cost
        self.d_cost_real = tf.nn.sigmoid_cross_entropy_with_logits(
            logits = logits,
            labels = tf.ones_like(logits)
        )
        
        self.d_cost_fake = tf.nn.sigmoid_cross_entropy_with_logits(
            logits = sample_logits,
            lables = tf.zeros_like(logits)
        )
        
        self.d_cost = tf.reduce_mean(self.d_cost_real) + tf.reduce_mean(self.d_cost_fake)
        
        self.g_cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits = sample_logits,
                labels = tf.ones_like(sample_logits)
            )
        )
        
        real_prediction = tf.cast(logits > 0, tf.float32)
        fake_prediction = tf.cast(sample_logits < 0, tf.float32)
        num_prediction = 2.0*batch_size
        num_correct = tf.reduce_sum(real_prediction) + tf.reduce_sum(fake_prediction)
        self.d_accuracy = num_correct/num_prediction
        
        #optimize
        self.d_params = [d for d in tf.trainable_variables() if t.name.startwith('d')]
        self.g_params = [g for g in tf.trainable_variables() if t.name.startwith('g')]
        
        self.d_trainop = tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(self.d_cost, var_list = self.d_params)
        self.g_trainop = tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(self.g_cost, var_list = self.g_params)
        
        self.initop = tf.global_variables_initializer()
        self.s = tf.InteractiveSession()
        self.s.run(self.initop)
    
    def build_discriminator(self, X, d_sizes):
        with tf.variable_scope('discriminator') as scope:
            
            #build conv layer
            self.d_convlayers = []
            mi = self.num_colors
            dim = self.img_length
            count = 0
            for mo, filter_size, stride, apply_batch_norm in d_sizes['conv_layers']:
                name = "convlayer_%s"%count
                count += 1
                
                layer = ConvLayer(name, mi, mo,apply_batch_norm, filter_size,stride,lrelu)
                self.d_convlayers.append(layer)
                mi = mo
                dim = int(np.ceil(float(dim)/stride))
                
            mi = mi*dim*dim
            #build dense layer
            self.d_denselayers = []
            for mo, apply_batch_norm in d_sizes['dense_layers']:
                name = "denselayer_%s"%count
                count += 1
                
                layer = DenseLayer(name,mi, mo,apply_batch_norm, lrelu)
                self.d_denselayers.append(layer)
                mi = mo
                
            #final logistic layer
            name = "denselayer_%s"%count
            
            self.d_finallayer = DenseLayer(name,mi,1,False,lambda x:x)
            
            #get logit
            logits = self.d_forward(X)
            return logits
    
    def d_forward(self,X, reuse = None, is_training = True):
        output = X
        for layer in self.d_convlayers:
            output = layer.forward(output,reuse,is_training)
        output = tf.contrib.layers.flatten(output)
        for layer in self.d_denselayers:
            output = layer.forward(output,reuse,is_training)
        logits = self.d_finallayer.forward(output,reuse,is_training)
        return logits
        
    
    def build_generator(self, Z, g_sizes):
        with tf.variable_scope("generator") as scope:
            dims = [self.img_length]
            dim = self.img_length
            for _, _, stride, _ in reversed(g_sizes['conv_layers']):
                dim = int(np.ceil(float(dim)/stride))
                dims.append(dim)
        #dims is backward
        dims = list(reversed(dims))
        self.g_dims = dims
        
        #dense layers
        mi = self.latent_dims
        self.g_denselayers = []
        count = 0
        for mo, apply_batch_norm in g_sizes['dense_layers']:
            name = "g_denselayer_%s"%count
            count += 1
            
            layer = DenseLayer(name, mi, mo, apply_batch_norm)
            self.g_denselayers.append(layer)
            mi = mo
        
        #final dense layer
        mo = g_sizes['projection']*dims[0]*dims[0]
        name = "g_denselayer_%s"%count
        layer = DenseLayer(name,mi,mo,g_sizes['bn_after_projection'])
        self.g_denselayers.append(layer)
        
        #fs-conv layer
        mi = g_sizes['projection']
        self.g_convlayesr= []
        
        num_relus = len(g_sizes['conv_layers']) - 1
        activation_functions = [tf.nn.relu] * num_relus + [g_sizes['output_activation']]
        
        for i in range(len(g_sizes['conv_layers'])):
            name = "fs_convlayer_%s"%i
            mo, filter_size,stride, apply_batch_norm = g_sizes['conv_layers'][i]
            f = activation_functions[i]
            output_shape = [self.batch_size,dims[i+1],dim[i+1],mo]
            layer = FractioanllyStridedConvLayer(
                name, mi, mo, output_shape, apply_batch_norm, filter_size,stride,f
            )
            self.g_convlayesr.append(layer)
            mi = mo
            
        self.g_sizes = g_sizes
        return self.g_forward(Z)
        
    
    def g_forward(self, Z, reuse = None, is_training = True):
        output = Z
        for layer in self.g_denselayers:
            output = layer.forward(output, reuse, is_training)
        
        output = tf.reshape(
            output,
            [-1, self.g_dims[0], self.g_dims[0], self.g_sizes['projection']]
        )
        
        if self.g_sizes['bn_after_projection']:
            output = tf.contrib.layers.batch_norm(
                output, 
                decay = 0.9,
                updates_collections = None,
                epsilon = 1e-5,
                scale = True,
                is_training = is_training,
                reuse = reuse,
                scope = 'bn_after_projection',
            )
            
        for layer in self.g_convlayesr:
            output = layer.forward(output, reuse, is_training)
        
        return output
    
    
    
    def fit(self,X):
        d_costs = []
        g_costs = []
        
        N = len(X)
        batch_num = int(N//batch_size)
        total_iter = 0
        
        for i in range(epi):
            print("epi: "+i)
            np.random.shuffle(X)
            for j in batch_num:
                t0 = datetime.now()
                batch = files2image(X[j*batch_size,(j+1)*batch_size])
            
                Z = np.random.uniform(-1,1,size = (batch_size,latent_dims))
                
                #train discriminator
                _, d_cost, d_acc = self.s.run(
                    (self.d_trainop,self.d_cost,self.d_accuracy),
                    feed_dict={self.X:X, self.Z:Z, self.batch_size:batch_size},
                )
                d_costs.append(d_cost)
                
                #train generator
                _, g_cost1 = self.s.run(
                    (self.g_trainop,self.g_cost),
                    feed_dict = {self.Z:Z,self.batch_size:batch_size},
                )
                _, g_cost2 = self.s.run(
                    (self.g_trainop, self.g_cost),
                    feed_dict = {self.Z:Z, self.batch_size:batch_size},
                )
                g_costs.append((g_cost1+g_cost2)/2)
                
                print("batch%d %d acc%f"%(i+1,j+1,d_acc))
                
                plt.clf()
                plt.plot(d_costs,label = "discriminator cost")
                plt.plot(g_costs, label = "generator cost")
                plt.legend()
                plt.savefig("cost.png")
                        

In [18]:
def get_mnist(limit = None):
    dt = pd.read_csv('C:\\Users\\Wei Guo\\Desktop\\data\\digit.csv')
    dt_value = dt.values
    X = dt_value[:,1:]
    Y = dt_value[:,0]
    if limit != None:
        X = X[:int(limit)]
        Y = Y[:int(limit)]
    return X,Y

In [19]:
def mnist():
    X,Y = get_mnist()
    X.reshape(len(X),28,28,1)
    dim = X.shape[1]
    colors = X.shape[-1]
    
    #set mnist layers
    d_sizes = {
        'conv_layers':[(2,5,2,False),(64,5,2,True)],
        'dense_layers':[(1024,True)],
    }
    g_sizes = {
        'z':100,
        'projection':128,
        'bn_after_projection':False,
        'conv_layers':[(128,5,2,True),(colors,5,2,False)],
        'dense_layers':[(1025,True)],
        'output_activation':tf.sigmoid,

    }
    
    gan = DCGAN(dim,colors,d_sizes,g_sizes)
    gan.fit(X)

In [20]:
mnist()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'